# Sudan Geospatial Analysis Dashboard

A modern, interactive geospatial analysis notebook powered by the **DuckDB Sudan Extension**.

This notebook combines data from **5 international APIs** (World Bank, WHO, FAO, UNHCR, ILO) with real **GADM v4.1 polygon boundaries** for Sudan's 18 states — all rendered with **Plotly** (dark theme, animated, interactive) and **Folium** (dark tiles, multi-layer, rich popups).

| Section | Visualization | Library |
|---------|--------------|--------|
| Choropleth Map | State area heatmap with labels | Plotly Mapbox |
| Population Race | Animated bar chart 1960–present | Plotly |
| Refugee Sankey | Flow diagram Sudan → asylum countries | Plotly |
| Regional Sunburst | Hierarchical area breakdown | Plotly |
| Health Dashboard | Maternal mortality + life expectancy | Plotly subplots |
| Agriculture Treemap | Top crops by production | Plotly |
| Multi-Layer Map | Dark tiles, popups, legend, plugins | Folium |
| Unemployment Radar | ILO data by sex & age group | Plotly polar |
| GDP Gauges | Per-capita gauge for 8 countries | Plotly indicators |
| Executive Dashboard | 3x2 combined summary | Plotly subplots |

## 1. Setup & Installation

In [1]:
%pip install duckdb==1.4.4 geopandas folium plotly shapely mapclassify branca kaleido --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00


> **Important:** After running the cell above, go to **Runtime → Restart runtime** before continuing.

In [2]:
import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import folium
from folium.plugins import Fullscreen, MiniMap, MeasureControl
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from shapely.geometry import shape

# ── Dark theme ──
pio.templates.default = "plotly_dark"

# ── Color palette ──
SUDAN_RED   = '#E63946'
NAVY        = '#1D3557'
TEAL        = '#2A9D8F'
SAND        = '#F4A261'
PURPLE      = '#9B59B6'
BG_DARK     = '#0E1117'
CARD_BG     = '#1B2838'
GRID_COLOR  = '#2D3748'
PALETTE     = [SUDAN_RED, TEAL, SAND, NAVY, PURPLE, '#E76F51', '#264653', '#606C38']

REGION_COLORS = {
    'Darfur':   SUDAN_RED,
    'Kordofan': SAND,
    'Central':  TEAL,
    'Eastern':  NAVY,
    'Northern': PURPLE
}

REGION_MAP = {
    'North Darfur': 'Darfur', 'South Darfur': 'Darfur', 'West Darfur': 'Darfur',
    'East Darfur': 'Darfur', 'Central Darfur': 'Darfur',
    'North Kordofan': 'Kordofan', 'South Kordofan': 'Kordofan', 'West Kordofan': 'Kordofan',
    'Khartoum': 'Central', 'Al Jazirah': 'Central', 'White Nile': 'Central',
    'Blue Nile': 'Central', 'Sennar': 'Central',
    'Kassala': 'Eastern', 'Al Qadarif': 'Eastern', 'Red Sea': 'Eastern',
    'River Nile': 'Northern', 'Northern': 'Northern'
}

def dark_layout(fig, title='', height=600):
    """Apply consistent dark styling to a Plotly figure."""
    fig.update_layout(
        title=dict(text=title, x=0.5, font=dict(size=20)),
        paper_bgcolor=BG_DARK,
        plot_bgcolor=CARD_BG,
        font=dict(color='#E2E8F0', family='Segoe UI, sans-serif'),
        height=height,
        margin=dict(t=80, b=60, l=60, r=40)
    )
    fig.update_xaxes(gridcolor=GRID_COLOR, zeroline=False)
    fig.update_yaxes(gridcolor=GRID_COLOR, zeroline=False)
    return fig

# ── Connect DuckDB ──
conn = duckdb.connect(config={'allow_unsigned_extensions': 'true'})
conn.execute("INSTALL httpfs; LOAD httpfs;")
conn.execute("SET custom_extension_repository = 'https://osman-geomatics93.github.io/duckdb-sudan-';")
conn.execute("INSTALL sudan; LOAD sudan;")

conn.sql("SELECT * FROM SUDAN_Providers()").show()
print('\nSudan extension loaded successfully!')

┌─────────────┬───────────────────────────────────┬────────────────────────┬────────────────────────────────────────────────────────────┬─────────────────────────────────────────┐
│ provider_id │               name                │        name_ar         │                        description                         │                base_url                 │
│   varchar   │              varchar              │        varchar         │                          varchar                           │                 varchar                 │
├─────────────┼───────────────────────────────────┼────────────────────────┼────────────────────────────────────────────────────────────┼─────────────────────────────────────────┤
│ worldbank   │ World Bank                        │ البنك الدولي           │ World Development Indicators and other World Bank datasets │ https://api.worldbank.org/v2/           │
│ who         │ World Health Organization         │ منظمة الصحة العالمية   │ Global Health Observato

## 2. Load Boundaries

The extension embeds real **GADM v4.1 MultiPolygon** boundaries for all 18 states. No network needed for geometries.

In [3]:
# Fetch states with bilingual names
states_df = conn.sql("""
    SELECT state_name, state_name_ar, iso_code, centroid_lon, centroid_lat, geojson
    FROM SUDAN_States()
""").df()

# Parse to GeoDataFrame
states_df['geometry'] = states_df['geojson'].apply(lambda g: shape(json.loads(g)))
gdf = gpd.GeoDataFrame(states_df, geometry='geometry', crs='EPSG:4326')
gdf = gdf.drop(columns=['geojson'])

# Add region and area
gdf['region'] = gdf['state_name'].map(REGION_MAP)
gdf_proj = gdf.to_crs(epsg=32636)
gdf['area_km2'] = gdf_proj.geometry.area / 1e6

# Country outline
country_df = conn.sql("SELECT country_name, geojson FROM SUDAN_Boundaries('country')").df()
country_df['geometry'] = country_df['geojson'].apply(lambda g: shape(json.loads(g)))
country_gdf = gpd.GeoDataFrame(country_df, geometry='geometry', crs='EPSG:4326')
country_gdf = country_gdf.drop(columns=['geojson'])

# Build GeoJSON FeatureCollection for Plotly choropleth
features = []
for _, row in gdf.iterrows():
    feat = {
        'type': 'Feature',
        'id': row['iso_code'],
        'properties': {
            'state_name': row['state_name'],
            'state_name_ar': row['state_name_ar'],
            'region': row['region'],
            'area_km2': round(row['area_km2'])
        },
        'geometry': row['geometry'].__geo_interface__
    }
    features.append(feat)
geojson_fc = {'type': 'FeatureCollection', 'features': features}

print(f'Loaded {len(gdf)} states, total area: {gdf["area_km2"].sum():,.0f} km\u00b2')
gdf[['state_name', 'state_name_ar', 'iso_code', 'region', 'area_km2']].head()

Loaded 18 states, total area: 1,883,712 km²


,state_name,state_name_ar,iso_code,region,area_km2
0,Khartoum,الخرطوم,SD-KH,Central,21129.046676
1,Al Jazirah,الجزيرة,SD-GZ,Central,24214.116036
2,Al Qadarif,القضارف,SD-GD,Eastern,64109.983971
3,Kassala,كسلا,SD-KA,Eastern,45687.989096
4,Red Sea,البحر الأحمر,SD-RS,Eastern,216869.521267


## 3. Interactive Choropleth Map (Plotly)

State area heatmap on dark Mapbox tiles with centroid labels and bilingual hover info.

In [4]:
# Choropleth colored by area
fig = px.choropleth_mapbox(
    gdf,
    geojson=geojson_fc,
    locations='iso_code',
    color='area_km2',
    color_continuous_scale='Turbo',
    hover_name='state_name',
    hover_data={
        'state_name_ar': True,
        'iso_code': True,
        'region': True,
        'area_km2': ':.0f'
    },
    mapbox_style='carto-darkmatter',
    center={'lat': 15.5, 'lon': 30.0},
    zoom=4.8,
    opacity=0.75,
    labels={'area_km2': 'Area (km\u00b2)'}
)

# Add centroid labels
fig.add_trace(go.Scattermapbox(
    lat=gdf['centroid_lat'],
    lon=gdf['centroid_lon'],
    mode='text',
    text=gdf['state_name'],
    textfont=dict(size=9, color='white', family='Segoe UI'),
    textposition='middle center',
    hoverinfo='skip',
    showlegend=False
))

fig.update_layout(
    title=dict(text='Sudan \u2014 State Area Choropleth', x=0.5, font=dict(size=22, color='#E2E8F0')),
    paper_bgcolor=BG_DARK,
    font=dict(color='#E2E8F0', family='Segoe UI'),
    height=700,
    margin=dict(t=60, b=20, l=20, r=20),
    coloraxis_colorbar=dict(
        title='Area (km\u00b2)',
        bgcolor=CARD_BG,
        bordercolor=GRID_COLOR,
        tickfont=dict(color='#E2E8F0')
    )
)
fig.show()

/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




## 4. Animated Population Race (Plotly)

An animated horizontal bar chart showing population growth across Sudan and its 7 neighbors from 1960 to present.

In [5]:
pop_df = conn.sql("""
    SELECT country_name, country, year, value
    FROM SUDAN_WorldBank('SP.POP.TOTL',
         countries := ['SDN','EGY','ETH','TCD','SSD','ERI','LBY','CAF'])
    WHERE value IS NOT NULL AND year >= 1960
    ORDER BY year, value
""").df()

pop_df['pop_millions'] = pop_df['value'] / 1e6
pop_df['label'] = pop_df['pop_millions'].apply(lambda x: f'{x:.1f}M')

# Assign consistent colors per country
countries = pop_df['country_name'].unique()
country_color_map = {c: PALETTE[i % len(PALETTE)] for i, c in enumerate(sorted(countries))}
# Override Sudan to always be red
for name in countries:
    if 'sudan' in name.lower() and 'south' not in name.lower():
        country_color_map[name] = SUDAN_RED

fig = px.bar(
    pop_df,
    x='pop_millions',
    y='country_name',
    orientation='h',
    animation_frame='year',
    color='country_name',
    color_discrete_map=country_color_map,
    text='label',
    labels={'pop_millions': 'Population (millions)', 'country_name': ''},
    range_x=[0, pop_df['pop_millions'].max() * 1.15]
)

fig.update_traces(textposition='outside', textfont_size=11)
fig.update_layout(
    showlegend=False,
    yaxis=dict(categoryorder='total ascending'),
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        x=0.05, y=1.12,
        buttons=[dict(label='\u25b6 Play', method='animate',
                      args=[None, dict(frame=dict(duration=300, redraw=True),
                                       fromcurrent=True)])]
    )]
)
dark_layout(fig, 'Population Race \u2014 Sudan & Neighbors (1960\u2013present)', height=550)
fig.show()

## 5. Refugee Flow Sankey Diagram (Plotly)

Visualizes the flow of Sudanese refugees from Sudan to top asylum countries using UNHCR data.

In [6]:
refugees_df = conn.sql("""
    SELECT year, country_asylum_name, value
    FROM SUDAN_UNHCR('refugees')
    WHERE value > 0
    ORDER BY year DESC, value DESC
""").df()

if len(refugees_df) > 0:
    latest_year = refugees_df['year'].max()
    top = refugees_df[refugees_df['year'] == latest_year].nlargest(12, 'value')

    # Build Sankey: Sudan (node 0) -> asylum countries (nodes 1..n)
    asylum_countries = top['country_asylum_name'].tolist()
    node_labels = ['Sudan'] + asylum_countries
    node_colors = [SUDAN_RED] + [PALETTE[i % len(PALETTE)] for i in range(len(asylum_countries))]

    max_val = top['value'].max()
    link_opacity = [max(0.3, 0.9 * v / max_val) for v in top['value']]
    link_colors = [f'rgba(230, 57, 70, {op})' for op in link_opacity]

    fig = go.Figure(go.Sankey(
        arrangement='snap',
        node=dict(
            pad=20,
            thickness=25,
            label=node_labels,
            color=node_colors,
            line=dict(color=GRID_COLOR, width=1)
        ),
        link=dict(
            source=[0] * len(asylum_countries),
            target=list(range(1, len(asylum_countries) + 1)),
            value=top['value'].tolist(),
            color=link_colors,
            hovertemplate='Sudan \u2192 %{target.label}<br>Refugees: %{value:,.0f}<extra></extra>'
        )
    ))

    dark_layout(fig, f'Sudanese Refugee Flows ({latest_year})', height=500)
    fig.show()
else:
    print('No refugee data available')

## 6. Regional Sunburst (Plotly)

Hierarchical view of Sudan's geography: Center → 5 Regions → 18 States, sized by area.

In [8]:
total_area = gdf['area_km2'].sum()
sunburst_df = gdf[['state_name', 'region', 'area_km2']].copy()
sunburst_df['pct'] = (sunburst_df['area_km2'] / total_area * 100).round(1)
sunburst_df['parent_label'] = 'Sudan'

fig = px.sunburst(
    sunburst_df,
    path=['parent_label', 'region', 'state_name'],
    values='area_km2',
    color='region',
    color_discrete_map=REGION_COLORS,
    hover_data={'area_km2': ':.0f', 'pct': ':.1f'}
)

fig.update_traces(
    textinfo='label+percent parent',
    insidetextorientation='radial',
    hovertemplate='<b>%{label}</b><br>Area: %{value:,.0f} km\u00b2<br>Share: %{percentParent:.1%}<extra></extra>',
    marker=dict(line=dict(color=BG_DARK, width=2))
)

dark_layout(fig, 'Sudan \u2014 Regional Area Breakdown', height=650)
fig.update_layout(margin=dict(t=80, b=20, l=20, r=20))
fig.show()

## 7. Health Dashboard (Plotly)

Maternal mortality ratio and life expectancy trends from the WHO Global Health Observatory.

In [9]:
# Fetch health data
mmr_df = conn.sql("""
    SELECT year, value FROM SUDAN_WHO('MDG_0000000026')
    WHERE value IS NOT NULL ORDER BY year
""").df()

le_df = conn.sql("""
    SELECT year, sex, value FROM SUDAN_WHO('WHOSIS_000001')
    WHERE value IS NOT NULL ORDER BY year, sex
""").df()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Maternal Mortality Ratio (per 100k live births)',
                    'Life Expectancy at Birth (years)'),
    horizontal_spacing=0.1
)

# Left: MMR with area fill
if len(mmr_df) > 0:
    fig.add_trace(go.Scatter(
        x=mmr_df['year'], y=mmr_df['value'],
        mode='lines+markers',
        line=dict(color=SUDAN_RED, width=3),
        marker=dict(size=7, color=SUDAN_RED),
        fill='tozeroy',
        fillcolor='rgba(230, 57, 70, 0.15)',
        name='MMR',
        hovertemplate='Year: %{x}<br>MMR: %{y:.0f}<extra></extra>'
    ), row=1, col=1)

# Right: Life expectancy by sex
sex_colors = {'Both sexes': TEAL, 'Male': NAVY, 'Female': SUDAN_RED}
if len(le_df) > 0:
    for sex in le_df['sex'].unique():
        data = le_df[le_df['sex'] == sex]
        color = sex_colors.get(sex, SAND)
        fig.add_trace(go.Scatter(
            x=data['year'], y=data['value'],
            mode='lines+markers',
            line=dict(color=color, width=2.5),
            marker=dict(size=6),
            name=sex,
            hovertemplate=f'{sex}<br>Year: %{{x}}<br>Life exp: %{{y:.1f}} yrs<extra></extra>'
        ), row=1, col=2)

dark_layout(fig, 'Sudan \u2014 Health Indicators (WHO)', height=450)
fig.update_layout(legend=dict(x=0.55, y=0.05, bgcolor='rgba(0,0,0,0)'))
fig.show()

## 8. Agriculture Treemap (Plotly)

Top crops by production volume from FAOSTAT, displayed as a treemap.

In [10]:
crops_df = conn.sql("""
    SELECT item, year, value, unit
    FROM SUDAN_FAO('QCL', 'production')
    WHERE value IS NOT NULL AND value > 0
    ORDER BY year DESC, value DESC
""").df()

if len(crops_df) > 0:
    latest_crop_year = crops_df['year'].max()
    top_crops = crops_df[crops_df['year'] == latest_crop_year].nlargest(15, 'value').copy()
    top_crops['label'] = top_crops.apply(
        lambda r: f"{r['item']}<br>{r['value']/1e3:.0f}k tonnes", axis=1
    )
    top_crops['parent'] = 'Sudan'

    fig = px.treemap(
        top_crops,
        path=['parent', 'item'],
        values='value',
        color='value',
        color_continuous_scale='YlOrRd',
        hover_data={'value': ':,.0f'},
        labels={'value': 'Production (tonnes)'}
    )

    fig.update_traces(
        textinfo='label+percent parent',
        textfont=dict(size=13, color='white', family='Segoe UI'),
        hovertemplate='<b>%{label}</b><br>Production: %{value:,.0f} tonnes<br>Share: %{percentParent:.1%}<extra></extra>',
        marker=dict(line=dict(color=BG_DARK, width=2))
    )

    dark_layout(fig, f'Sudan \u2014 Top Crops by Production ({latest_crop_year})', height=550)
    fig.update_layout(margin=dict(t=80, b=20, l=20, r=20))
    fig.show()
else:
    print('No crop data available')

## 9. Multi-Layer Folium Map

Dark-themed interactive map with:
- **Layer 1:** Region-colored state polygons (45% opacity, gold highlight on hover)
- **Layer 2:** Country outline (gold dashed line)
- **Layer 3:** Centroid labels and circle markers
- Fullscreen, MiniMap, and MeasureControl plugins
- Rich HTML popups with styled tables
- Dark semi-transparent legend

In [11]:
# ── Base map ──
m = folium.Map(
    location=[15.5, 30.0],
    zoom_start=6,
    tiles='cartodbdark_matter'
)

# ── Layer 1: State boundaries ──
states_layer = folium.FeatureGroup(name='State Boundaries', show=True)

for _, row in gdf.iterrows():
    region_color = REGION_COLORS.get(row['region'], '#888')

    popup_html = f"""
    <div style="font-family:'Segoe UI',sans-serif; width:220px;
                background:{CARD_BG}; color:#E2E8F0; padding:12px;
                border-radius:8px; border:1px solid {GRID_COLOR};">
        <h3 style="margin:0 0 4px 0; color:{region_color};">{row['state_name']}</h3>
        <h3 style="margin:0 0 8px 0; direction:rtl; color:{region_color};">{row['state_name_ar']}</h3>
        <table style="width:100%; font-size:12px; border-collapse:collapse;">
            <tr style="border-bottom:1px solid {GRID_COLOR};">
                <td style="padding:4px 0; color:#94A3B8;">ISO Code</td>
                <td style="padding:4px 0; text-align:right; font-weight:bold;">{row['iso_code']}</td>
            </tr>
            <tr style="border-bottom:1px solid {GRID_COLOR};">
                <td style="padding:4px 0; color:#94A3B8;">Region</td>
                <td style="padding:4px 0; text-align:right;">
                    <span style="background:{region_color}; color:white; padding:2px 8px;
                                 border-radius:10px; font-size:11px;">{row['region']}</span>
                </td>
            </tr>
            <tr>
                <td style="padding:4px 0; color:#94A3B8;">Area</td>
                <td style="padding:4px 0; text-align:right; font-weight:bold;">{row['area_km2']:,.0f} km\u00b2</td>
            </tr>
        </table>
    </div>
    """

    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda x, c=region_color: {
            'fillColor': c, 'color': '#E2E8F0',
            'weight': 1, 'fillOpacity': 0.45
        },
        highlight_function=lambda x: {
            'weight': 3, 'color': '#FFD700', 'fillOpacity': 0.65
        },
        popup=folium.Popup(popup_html, max_width=260),
        tooltip=folium.Tooltip(
            f"<b>{row['state_name']}</b> ({row['state_name_ar']})",
            style='background-color:#1B2838;color:#E2E8F0;border:1px solid #2D3748;'
                  'border-radius:4px;padding:6px;font-family:Segoe UI;'
        )
    ).add_to(states_layer)

states_layer.add_to(m)

# ── Layer 2: Country outline ──
outline_layer = folium.FeatureGroup(name='Country Outline', show=True)
folium.GeoJson(
    country_gdf.geometry.iloc[0].__geo_interface__,
    style_function=lambda x: {
        'fillColor': 'transparent', 'color': '#FFD700',
        'weight': 2.5, 'dashArray': '8 4', 'fillOpacity': 0
    }
).add_to(outline_layer)
outline_layer.add_to(m)

# ── Layer 3: Centroid labels + circle markers ──
labels_layer = folium.FeatureGroup(name='State Labels', show=True)

for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row['centroid_lat'], row['centroid_lon']],
        radius=4,
        color='white', fill=True, fill_color='white', fill_opacity=0.9,
        weight=1
    ).add_to(labels_layer)

    folium.Marker(
        location=[row['centroid_lat'], row['centroid_lon']],
        icon=folium.DivIcon(html=f"""
            <div style="font-size:9px; font-weight:bold; color:white;
                        text-shadow:0 0 4px #000, 0 0 8px #000;
                        white-space:nowrap; transform:translate(-50%,-20px);">
                {row['state_name']}
            </div>
        """)
    ).add_to(labels_layer)

labels_layer.add_to(m)

# ── Plugins ──
Fullscreen(position='topright').add_to(m)
MiniMap(tile_layer='cartodbdark_matter', toggle_display=True, position='bottomright').add_to(m)
MeasureControl(position='topleft', primary_length_unit='kilometers').add_to(m)

# ── Layer control ──
folium.LayerControl(position='topright', collapsed=False).add_to(m)

# ── Dark legend ──
legend_html = '''
<div style="position:fixed; bottom:30px; left:30px; z-index:1000;
            background:rgba(27,40,56,0.92); padding:14px 18px;
            border-radius:8px; border:1px solid #2D3748;
            font-family:'Segoe UI',sans-serif; font-size:12px; color:#E2E8F0;">
    <div style="font-weight:bold; font-size:13px; margin-bottom:8px; color:#FFD700;">Regions</div>
    <div style="margin:4px 0;"><span style="background:#E63946; width:14px; height:14px;
         display:inline-block; border-radius:3px; margin-right:8px; vertical-align:middle;"></span>Darfur</div>
    <div style="margin:4px 0;"><span style="background:#F4A261; width:14px; height:14px;
         display:inline-block; border-radius:3px; margin-right:8px; vertical-align:middle;"></span>Kordofan</div>
    <div style="margin:4px 0;"><span style="background:#2A9D8F; width:14px; height:14px;
         display:inline-block; border-radius:3px; margin-right:8px; vertical-align:middle;"></span>Central</div>
    <div style="margin:4px 0;"><span style="background:#1D3557; width:14px; height:14px;
         display:inline-block; border-radius:3px; margin-right:8px; vertical-align:middle;"></span>Eastern</div>
    <div style="margin:4px 0;"><span style="background:#9B59B6; width:14px; height:14px;
         display:inline-block; border-radius:3px; margin-right:8px; vertical-align:middle;"></span>Northern</div>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

m

## 10. Unemployment Radar Chart (Plotly)

ILO unemployment rates by sex across age groups, displayed as overlapping radar polygons.

In [12]:
# Try to fetch ILO unemployment data
try:
    ilo_df = conn.sql("""
        SELECT indicator_name, sex, age_group, year, value
        FROM SUDAN_ILO('UNE_DEAP_SEX_AGE_RT')
        WHERE value IS NOT NULL
        ORDER BY year DESC
    """).df()
except Exception:
    ilo_df = pd.DataFrame()

if len(ilo_df) > 0:
    latest_ilo_year = ilo_df['year'].max()
    radar_data = ilo_df[ilo_df['year'] == latest_ilo_year].copy()

    fig = go.Figure()

    sex_styles = {
        'Total': dict(color=TEAL, fill='rgba(42,157,143,0.2)'),
        'Male':  dict(color=NAVY, fill='rgba(29,53,87,0.2)'),
        'Female': dict(color=SUDAN_RED, fill='rgba(230,57,70,0.2)')
    }

    for sex_val in radar_data['sex'].unique():
        subset = radar_data[radar_data['sex'] == sex_val]
        style = sex_styles.get(sex_val, dict(color=SAND, fill='rgba(244,162,97,0.2)'))

        categories = subset['age_group'].tolist()
        values = subset['value'].tolist()
        # Close the polygon
        categories += [categories[0]]
        values += [values[0]]

        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            name=sex_val,
            line=dict(color=style['color'], width=2.5),
            fill='toself',
            fillcolor=style['fill'],
            hovertemplate=f'{sex_val}<br>%{{theta}}: %{{r:.1f}}%<extra></extra>'
        ))

    fig.update_polars(
        bgcolor=CARD_BG,
        angularaxis=dict(gridcolor=GRID_COLOR, linecolor=GRID_COLOR),
        radialaxis=dict(gridcolor=GRID_COLOR, linecolor=GRID_COLOR,
                        ticksuffix='%')
    )
    dark_layout(fig, f'Unemployment by Sex & Age Group ({latest_ilo_year})', height=550)
    fig.update_layout(legend=dict(x=0.85, y=0.95, bgcolor='rgba(0,0,0,0)'))
    fig.show()
else:
    print('ILO unemployment data not available \u2014 the API may be temporarily unreachable.\n'
          'The radar chart will render when data is accessible.')

ILO unemployment data not available — the API may be temporarily unreachable.
The radar chart will render when data is accessible.


## 11. GDP Per Capita Gauges (Plotly)

Gauge indicators for GDP per capita across Sudan and its 7 neighbors. Three color zones: red (low), amber (medium), green (high). Sudan is highlighted in red.

In [13]:
gdp_df = conn.sql("""
    SELECT country_name, country, year, value
    FROM SUDAN_WorldBank('NY.GDP.PCAP.CD',
         countries := ['SDN','EGY','ETH','TCD','SSD','ERI','LBY','CAF'])
    WHERE value IS NOT NULL
    ORDER BY year DESC
""").df()

if len(gdp_df) > 0:
    # Get latest available value per country
    latest_gdp = gdp_df.groupby('country_name').first().reset_index()
    latest_gdp = latest_gdp.sort_values('value', ascending=False)

    n = len(latest_gdp)
    cols = 4
    rows = (n + cols - 1) // cols

    fig = make_subplots(
        rows=rows, cols=cols,
        specs=[[{'type': 'indicator'}] * cols for _ in range(rows)],
        vertical_spacing=0.15,
        horizontal_spacing=0.08
    )

    max_gdp = latest_gdp['value'].max()
    gauge_max = int(np.ceil(max_gdp / 1000) * 1000)

    for i, (_, row) in enumerate(latest_gdp.iterrows()):
        r = i // cols + 1
        c = i % cols + 1
        is_sudan = 'sudan' in row['country_name'].lower() and 'south' not in row['country_name'].lower()

        fig.add_trace(go.Indicator(
            mode='gauge+number',
            value=row['value'],
            number=dict(prefix='$', valueformat=',.0f', font=dict(size=18)),
            title=dict(text=row['country_name'], font=dict(size=13)),
            gauge=dict(
                axis=dict(range=[0, gauge_max], tickprefix='$', tickfont=dict(size=9)),
                bar=dict(color=SUDAN_RED if is_sudan else TEAL, thickness=0.7),
                bgcolor=CARD_BG,
                bordercolor=GRID_COLOR,
                steps=[
                    dict(range=[0, gauge_max * 0.33], color='rgba(230,57,70,0.15)'),
                    dict(range=[gauge_max * 0.33, gauge_max * 0.66], color='rgba(244,162,97,0.15)'),
                    dict(range=[gauge_max * 0.66, gauge_max], color='rgba(42,157,143,0.15)')
                ],
                threshold=dict(
                    line=dict(color='#FFD700', width=3),
                    thickness=0.8,
                    value=row['value']
                )
            )
        ), row=r, col=c)

    dark_layout(fig, 'GDP Per Capita \u2014 Sudan & Neighbors (USD)', height=rows * 280)
    fig.update_layout(margin=dict(t=80, b=30, l=40, r=40))
    fig.show()
else:
    print('GDP per capita data not available')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## 12. Combined Executive Dashboard (Plotly)

A single-screen 3\u00d72 summary combining population, refugees, health, and agriculture.

In [14]:
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Population Over Time', 'Sudanese Refugees by Destination',
        'Maternal Mortality Ratio', 'Life Expectancy at Birth',
        'Top Crops (latest year)', 'GDP Per Capita'
    ),
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# ── (1,1) Population lines ──
if len(pop_df) > 0:
    for country in sorted(pop_df['country_name'].unique()):
        data = pop_df[pop_df['country_name'] == country].sort_values('year')
        is_sudan = 'sudan' in country.lower() and 'south' not in country.lower()
        fig.add_trace(go.Scatter(
            x=data['year'], y=data['pop_millions'],
            mode='lines', name=country,
            line=dict(color=SUDAN_RED if is_sudan else None,
                      width=3 if is_sudan else 1.5),
            showlegend=False,
            hovertemplate=f'{country}<br>%{{x}}: %{{y:.1f}}M<extra></extra>'
        ), row=1, col=1)

# ── (1,2) Refugee bar chart ──
if len(refugees_df) > 0:
    latest_ref_year = refugees_df['year'].max()
    top_ref = refugees_df[refugees_df['year'] == latest_ref_year].nlargest(8, 'value')
    top_ref = top_ref.sort_values('value', ascending=True)
    fig.add_trace(go.Bar(
        y=top_ref['country_asylum_name'], x=top_ref['value'],
        orientation='h',
        marker_color=SUDAN_RED,
        showlegend=False,
        hovertemplate='%{y}: %{x:,.0f}<extra></extra>'
    ), row=1, col=2)

# ── (2,1) MMR ──
if len(mmr_df) > 0:
    fig.add_trace(go.Scatter(
        x=mmr_df['year'], y=mmr_df['value'],
        mode='lines+markers', line=dict(color=SUDAN_RED, width=2),
        marker=dict(size=5),
        fill='tozeroy', fillcolor='rgba(230,57,70,0.1)',
        showlegend=False,
        hovertemplate='%{x}: %{y:.0f}<extra></extra>'
    ), row=2, col=1)

# ── (2,2) Life expectancy ──
if len(le_df) > 0:
    sex_c = {'Both sexes': TEAL, 'Male': NAVY, 'Female': SUDAN_RED}
    for sex in le_df['sex'].unique():
        data = le_df[le_df['sex'] == sex]
        fig.add_trace(go.Scatter(
            x=data['year'], y=data['value'],
            mode='lines+markers', name=sex,
            line=dict(color=sex_c.get(sex, SAND), width=2),
            marker=dict(size=4),
            showlegend=False,
            hovertemplate=f'{sex}<br>%{{x}}: %{{y:.1f}} yrs<extra></extra>'
        ), row=2, col=2)

# ── (3,1) Top crops bar ──
if len(crops_df) > 0:
    lcy = crops_df['year'].max()
    tc = crops_df[crops_df['year'] == lcy].nlargest(8, 'value').sort_values('value', ascending=True)
    fig.add_trace(go.Bar(
        y=tc['item'], x=tc['value'] / 1e3,
        orientation='h',
        marker_color=TEAL,
        showlegend=False,
        hovertemplate='%{y}: %{x:.0f}k tonnes<extra></extra>'
    ), row=3, col=1)

# ── (3,2) GDP per capita bar ──
if len(gdp_df) > 0:
    lg = gdp_df.groupby('country_name').first().reset_index().sort_values('value', ascending=True)
    bar_colors = [SUDAN_RED if ('sudan' in c.lower() and 'south' not in c.lower())
                  else NAVY for c in lg['country_name']]
    fig.add_trace(go.Bar(
        y=lg['country_name'], x=lg['value'],
        orientation='h',
        marker_color=bar_colors,
        showlegend=False,
        hovertemplate='%{y}: $%{x:,.0f}<extra></extra>'
    ), row=3, col=2)

dark_layout(fig, 'Sudan \u2014 Executive Dashboard', height=900)
fig.update_layout(margin=dict(t=80, b=40, l=80, r=40))
fig.show()

## 13. Export

Save all analysis outputs as portable files.

In [15]:
import os

out_dir = 'sudan_exports'
os.makedirs(out_dir, exist_ok=True)

# GeoJSON
gdf.to_file(f'{out_dir}/sudan_states.geojson', driver='GeoJSON')
print('Exported: sudan_states.geojson')

# GeoPackage
gdf.to_file(f'{out_dir}/sudan_states.gpkg', driver='GPKG')
print('Exported: sudan_states.gpkg')

# Summary CSV
summary = gdf[['state_name', 'state_name_ar', 'iso_code', 'region', 'area_km2',
               'centroid_lon', 'centroid_lat']].copy()
summary.to_csv(f'{out_dir}/sudan_states_summary.csv', index=False)
print('Exported: sudan_states_summary.csv')

# Folium map HTML
m.save(f'{out_dir}/sudan_dark_map.html')
print('Exported: sudan_dark_map.html')

# Plotly dashboard HTML
fig.write_html(f'{out_dir}/sudan_dashboard.html', include_plotlyjs='cdn')
print('Exported: sudan_dashboard.html')

print(f'\nAll exports saved to {out_dir}/')

# Auto-download in Colab
try:
    from google.colab import files
    for f in os.listdir(out_dir):
        files.download(f'{out_dir}/{f}')
except ImportError:
    pass

Exported: sudan_states.geojson
Exported: sudan_states.gpkg
Exported: sudan_states_summary.csv
Exported: sudan_dark_map.html
Exported: sudan_dashboard.html

All exports saved to sudan_exports/


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>